# RNA-seq入门实战（二）：上游数据的比对计数——Hisat2+ featureCounts 与 Salmon

> 本节概览：
> 1. `hisat2` + `featureCounts`:
> 获取hisat2索引文件，`hisat2`比对和`samtools`格式转化，`featureCounts`计数得到`counts`表达矩阵
> 2. `Salmon`:
> `salmon index` 用`cdna.fa`建立索引，`salmon quant`对`clean fastq`文件直接进行基因定量
> 3. 获取`ensembl_id`或`transcript_id`转化的对应文件

## 一、hisat2 + featureCounts
1. 获取hisat2比对索引文件
index官网下载地址[`Download | HISAT2 (daehwankimlab.github.io)`](https://link.zhihu.com/?target=http%3A//daehwankimlab.github.io/hisat2/download/%23h-sapiens)，下载并解压所需的 `mm10` 或 `grcm38` 的`index`文件

```shell
mkdir -p ~/reference/index/hisat/
cd ~/reference/index/hisat/
wget https://genome-idx.s3.amazonaws.com/hisat/mm10_genome.tar.gz
tar -zxvf *tar.gz 
rm *tar.gz
## 或
conda install hisat2
conda install -c bioconda subread
```

2. `hisat2`比对和`samtools`转化格式

**非常重要的步骤‼️**

先用`hisat2`比对基因组得到sam文件，再用`samtools sort`将sam文件格式转化与排序为`bam`文件（bam相当于二进制版的sam），之后`samtools index`建立索引（用于后续IGV内可视化），最后`samtools flag` 统计文件比对情况保存在文本中。其中`samtools index`与`samtools flag`为非必须步骤，可略过。`sam`相当于是中间文件比较占存储空间，可在转化为bam后便删除。


### hisat2 参数

hisat2主要参数:
- -x 参考基因组索引文件的前缀。注意最后是 **/genome**
- -1 双端䇷序结果的第一个文件。若有多组数据,使用逗号将文件分隔， Reads的长度可以不一致。
- -2 双觊䇣序结果的第二个文件。若有多组数据,使用逗号将文件分隔,并且文件顺序要和-1参数对应. Reads的长度可以不一致。
- -S 指定输出的SAM文件。
- -t 输出time
- -p 线程使用设置
- -U 单端数据文件.若有多组数据,使用逗号将文件分隔.可以和-1、-2参数同时使用. Reads的长度可以不一致。
- -sra-acc 输入`SRA`登录号，比如SRR12207279，SRR12207280，若有多组数据,使用逗号将文件分隔，HISAT将启动下载并识别数据类型，进行对比




## samtools
```shell
conda install samtools
```

### 参数
#### 3.1 samtools sort 进行格式转化和排序

```shell
ls *.sam | while read id ;do samtools sort -@ 15 -o ~/wk/align/$(basename $id '.sam')_sort.bam $id ;
done

```
- -@ INT 设置排序和压缩线程个数，默认是单线程
- -o FILE 设置最终排序后的输出文件名
- -I INT 设置输出文件压缩等级。0-9，0是不压缩，9是压缩等级最高。不设置此参数时，使用默认压缩等级；
- -m INT 设置每个线程运行时的内存大小，可以使用K,M和G表示内存大小
- -n 设置按照read名称进行排序
- -O FORMAT 设置最终输出的文件格式，可以是bam,sam或者cram，默认为bam.
- T PREFIX  设置临时文件前缀


less命令可以正常查看sam文件，但是不能正常查看bam文文件，因为bam文文件是二进制文件，所以需要使用

`samtools view *.bam|less -s`  查看bam文件     

`samtools view SRR1039508_sort.bam chrX:24001837-24045303|less -SN`  查看bam文件特定区域

#### 3.2 samtools index 建立索引

**步骤**：必须对`bam`文件进行默认情况下的排序后，才能进行index。否则会报错。建立索引后将产生后缀为`.bai`的文件，用于快速的随机处理。很多情况下需要有`bai`文件的存在，特别是显示序列比对情况下。比如`samtools`的`tview`命令就需要；`gbrowse2`显示 `reads`的比对图形的时候也需要。`IGV`显示比对情况也需要。

```shell
ls *.bam | xargs -i samtools index -@ 15 {}
```
- -b  创建一个BAI索引。当不使用格式选项时，这是当前的默认设置
- -c  创建CSI索引。默认情况下 ，索引的最小间隔大小为2^14，与BAI格式使用的固定值相同
- m INT # 创建CSI索引，最小格式大小2^INT
- -@, --threads INT 除了主线程[0]外，要使用的输入/输出压缩线程数


基因组可视化：`samtools tview  *.bam`  查看染色体区域的比对情况

使用工具IGV  `integrative Genomics Viewer`



### 3.3 samtools flagstat 统计文件比对情况 并保存在文本中.flagstat

```shell
ls *.bam | while read id ; do samtools flagstat -@ 15 $id > $(basename $id '.bam').flagstat ; done
```
- -@ INT # 设置读取文件时要使用的额外线程数。
- -O FORMAT # 设置输出格式。FORMAT可以设置为'default'， 'json'或'tsv'来选择默认的，json或标签分隔值输出格式，如果不使用此选项，将选择飘认格式。

```shell
ls *stat |while read id; do echo $id; sed "s/5)/5) \n/g" $id; done
# 展示`flagstat`文件，并显示出文件名，不同文件间空行（即把最后字符后添加到换行符）
```

`multiqc ./`  生成网页版的统计情况

绘制成excel 表格展示统计情况:

```shell
cat *.flagstat |awk '{print $1}'|paste ------------   

# 按照每行13个数字方式排列flagstat数据
```


`ctrl+c` 复制到excel表格，表格右下角选择 '使用文本导入向导' 将数据分割到小格中，复制粘贴进行转置


```shell
cat SRR1039508_sort.flagstat |awk -F '+ 0 ' '{print $2}'
# 输出文本数据解释的部分
# 粘贴至excel表中做成表格，保存为mytest文件
```

### shell 脚本
`vim  3_align2sam2bam_hisat2.sh`
```shell

############################
echo -e  " \n \n \n 333#  Align !!! hisat2 !!!\n \n \n "
date
########set#### ###set#### ###set####   
index='/home/reference/index/hisat/mm10/genome'

mkdir  -p   /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/align/flag
cd /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/align/
pwd
cat gene_process_for_python/data/idname | while read id
do
         echo "333#  ${id} ${id} ${id}  is on the hisat2 Working !!!"
################ paired ###############################         
            hisat2 -t -p 12 -x  $index \
       -1 /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/raw/clean/${id}_*1*gz  \
       -2  /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/raw/clean/${id}_*2*gz  -S  ${id}.sam
######################################################
################Single################################
#              hisat2 -t -p 12 -x  $index  \
#                     -U /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/raw/clean/${id}_trimmed.fq.gz \
#                     -S  ./${id}.sam  
########################################################           

### sam2bam and remove sam ###
    echo -e  " ${id} sam2bam and remove sam   "
    samtools sort -@ 12 -o  /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/align/${id}_sorted.bam  ${id}.sam
    rm ${id}.sam
done

#### samtools index and flagstat ####
echo -e  " \n \n \n samtools index and flagstat \n  " 
cd   /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/align/flag
pwd
#ls /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/align/*.bam | xargs  -i  samtools index  -@  12  {}    
ls /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/align/*.bam  | while read id ;\
do
        samtools flagstat -@ 12 $id > $(basename $id '.bam').flagstat
done
multiqc ./

echo -e " \n \n \n  333# ALL  Work Done !!!\n \n \n "
date

```

`nohup bash 3_align2sam2bam_hisat2.sh >log_3 2>&1 &`


`grep alignment  log_3`       

比对结果如下：




## 3. featureCounts 计数得到counts表达矩阵
计数首先要获取gtf注释文件 **，注意要和hisat2的index文件的基因组版本相对应** ，如本次为`mm10`，则`gtf`文件也必须为`mm10`或`grcm38`。 研究人和鼠推荐用`gencode数据库`的文件`GENCODE` ，比较常用的还有UCSC的`refGene.gtf`文件，下载地址在https://hgdownload.soe.ucsc.edu/ （若想下载其他`gtf`文件则将网址中`mm10`替换即可，如`hg38`）。

`featureCounts`详细使用方法见转录组定量可以用替换[`featureCounts`代替`HTSeq-count (qq.com)`](https://link.zhihu.com/?target=https%3A//mp.weixin.qq.com/s%3F__biz%3DMzAxMDkxODM1Ng%3D%3D%26mid%3D2247485726%26idx%3D1%26sn%3Deab040119aa041d5cdcd6e1c81179455%26scene%3D21%23wechat_redirect)，常用参数如下：

### featureCounts的使用

```shell
Usage: featureCounts [options] -a <annotation_file> -o <output_file> input_file1 [input_file2]

```
生成counts文档，同时将命令运行结果输出结果日志（1为标准输出， 2 为失败输出）

```shell
featureCounts -T 15 -p -a ~ref/ucsc/hg19.refGene.gtf.gz -o counts.txt  ~/wk/align/*.bam  1> counts.log  2>&1

multiqc ./ # 生成网页版统计情况
```

- -a 输入GTF/GFF基因组注释文件
- -p 这个参数是针对paired-end数据
- -F指定-a注释文件的格式，默认是GTF
- -g 从注释文件中提取Meta-features信息用于read count, 默认是gene_id
- -t 跟-g—样的意思，其是默认将exon作为一个feature
- -o 输出文件
- -T 多线程数


`vim 4_counts_featurecounts.sh`


```shell
###########################################
echo -e "\n \n \n444# Count #featureCounts !!! \n \n \n"
date
#####set####set###set
gtf='/slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/gencode.vM25.chr_patch_hapl_scaff.annotation.gtf'
#gtf='/home/reference/gtf/UCSC/mm10.refGene.gtf.gz'

mkdir  -p  /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/counts
cd  /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/counts
pwd
######## single##########################################################################
#featureCounts -T 12 -a $gtf  -o counts.txt  ~/test/align/*.bam        
#######paired###########################################################################
featureCounts -T  12  -p  -a  $gtf  -o  counts.txt  /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/align/*.bam
#######################################################################################            
###生成网页版统计情况
multiqc ./

echo -e " \n \n \n ALL WORK DONE !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  \n "
date

```


`nohup bash 4_counts_featurecounts.sh >log_4 2>&1 &`

查看计数的网页版统计情况，匹配率在71-80%左右，还可以
<image src="../image/19.png">

运行结果保存在counts文件夹下，里面的counts.txt就是我们下游分析所需要的文件啦

## 二、Salmon——直接对基因进行定量的工具

与hisat2不同，salmon不经过比对计数步骤而是直接对基因进行定量，如果不研究新转录本，用salmon方法可以更快更方便得到基因定量信息。

```shell
conda install salmon -c bioconda 
```


salmon相比含有hisat2和STAR等软的RNA-seq流程而言，速庭更快，这是因为该软件基于转录组序列reference（也即是cDNA序
列）并且基于k mert比对原理。
通常如果是研究RNA-seq过程基因发生了何种变化且不需知道新转录本，可以直接使用salmon获取转录本的丰富信息。
salmon是不基于比对计数而直接对基因进行定量的工具，适用于转录组、宏基因组等的分析。見原理摡括来讲是通过构建统计模型来推测已经注释的转录本呈现出什么表达模式时我们才会界序产生当前的FASTQ数据。其优势是：
- 定量时考虑到不同样品中基因长度的改变 (比如不同isoform目的使用)
- 速度快、需要的计算资涪和存㒂资源小
- 敏感性高，不会丢弃目百到多个基因同源区域的reads
- 可以直接校正GC-bias
- 自动判断文库类型

### 1. 建立salmon索引
先下载参考转录本序列cDNA.fa文件，在ensembl官网选择相应文件 [Index of /pub/release-102/fasta/mus_musculus/cdna/ (ensembl.org)](https://link.zhihu.com/?target=http%3A//ftp.ensembl.org/pub/release-102/fasta/mus_musculus/cdna/)，使用salmon index 建立索引文件，`salmon index`常用参数：

- -p 线程
- -t 接参考转录本序列
- -i 设置建立索引的位置
- -d 接decoy 件

```shell
mkdir ~/reference/index/salmon/grcm38
cd ~/reference/index/salmon/grcm38
salmon index -p 12 -t  ~/reference/ensembl/grcm38.cdna.fa.gz  -i  ./
```


### salmon定量基因
使用`salmon quant`命令对`clean fastq`文件直接进行基因定量，主要参数如下：

- -i 接之前生成的`salmon index`
- -p 线程
- -I [-libType] 描述库的格式字符串，-I A 设置软件自动判断库类型
- -r [-unmatedReads] 包含末配对读取的文件列表 (如单头读取)
- -1 [-mates1] 包含 #1 mates的文件q
- -2 [-mates2] 包含 #2 mates的文件
- -o           设置输出结果的文件名
- `-g /--geneMap`  #提供转录本与基因之间对应关系的文件。
 #如果提供了该文件，`Salmon`将会输出文件：`quant.sf` 和 `quant.genes.sf`。            
 #其中quant.genes.sf 文件中含有对基因表达水平的估计。               
 #提供的文件可以是GTF文件，也可是由TAB分隔符分隔文件，文件中的每一行含转录本名称和对应的基因名。              
 #以'.gtf', '.gff' 或 '.gff3'结尾的文件名均被当做GTF格式做处理解析。              
 #以其他字符结尾的被当做由TAB分隔符分隔的简单文件格式做处理解析。           



`vim 3333_salmon.sh`

```shell
###################################################
echo -e  '\n \n \n ### salmon quant is Working !!! \n \n'
###set##set###set#############
index="/home/reference/index/salmon/grcm38/"

mkdir /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/salmon
cd /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/salmon
pwd
cat /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/idname | while read id 
do
            echo " '\n  !!!!!! Processing sample $id !!!!! '\n" 
########single#############################################
# salmon quant -i $index -l A \
#             -r ~/test/clean/$(basename $id)_trimmed.fq.gz \
#             -p 12  -o  $(basename $id)_quant
#######paired#############################################
salmon quant -i $index -l A \
 -1 /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/clean/${id}_*1*gz \
 -2 /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/clean/${id}_*2*gz  \
 -p 12  --output ${id}_quant
###############################################################     
done

multiqc ./

echo -e " \n \n \n !!!!ALL WORK DONE !!!!!!!!!!!!!!!!!!!!! \n"
date

```

`nohup bash 3333_salmon.sh  >log_333salmon  2>&1 &`

运行结果存放在`salmon`文件夹下，里面的`quant.sf`即为下游分析所需要的文件

## 三、获取基因ID转化的对应文件

由于本次使用的为gencode或ensembl的gtf与cdna文件，因此最后得到的为ensembl_id (gene_id)和 transcript_id，形式为：ENSMUSG00000000001.1 ，而我们下游常用gene symbol进行展示，因此还需要从gtf注释文件中获取ensembl_id 、transcript_id与gene symbol的对应关系文件。 方法如下：

`vim gtf_geneid2symbol_gencode.sh`

```shell
#提取gtf注释文件中gene_id等与gene_name的对应关系,便于下游id转换
gtf="gencode.vM25.chr_patch_hapl_scaff.annotation.gtf"

### gene_id to gene_name
grep 'gene_id' $gtf | awk -F 'gene_id \"' '{print $2}' |awk -F '\"' '{print $1}' >gene_id_tmp
grep 'gene_id' $gtf | awk -F 'gene_name \"' '{print $2}' |awk -F '\"' '{print $1}' >gene_name_tmp
paste gene_id_tmp gene_name_tmp >last_tmp
uniq last_tmp >g2s_vm25_gencode.txt
rm *_tmp

### transcript_id to gene_name
grep 'transcript_id' $gtf | awk -F 'transcript_id \"' '{print $2}' |awk -F '\"' '{print $1}' >gene_id_tmp
grep 'transcript_id' $gtf | awk -F 'gene_name \"' '{print $2}' |awk -F '\"' '{print $1}' >gene_name_tmp
paste gene_id_tmp gene_name_tmp >last_tmp
uniq last_tmp >t2s_vm25_gencode.txt
rm *_tmp

```

`bash gtf_geneid2symbol_gencode.sh`

所得文件如下所示

<image src="../image/20.png">

上游流程到此就结束了，将最后得到的`counts`文件夹与`g2s_vm25_gencode.txt` 或 `salmon`文件夹与`t2s_vm25_gencode.txt`下载到本地就可以愉快地进行下游分析了